In [1]:
import sys 
sys.path.append("..") 
from collections import Counter
import json
import pandas as pd

from utils import data_helper,result_helper

In [2]:
kb_flag=7

ds3_path='../data/train/unlabeled_data_labeled_v1.csv'
ds7_path='../data/train/labeled_data_processed.csv'

kb3_save_path='../data/knowledgebase-3.json'
kb7_save_path='../data/knowledgebase-7.json'

if kb_flag==7:
    ds_path=ds7_path
    kb_save_path=kb7_save_path
else:
    ds_path=ds3_path
    kb_save_path=kb3_save_path


zero_shot_df=pd.read_csv(ds_path)

In [3]:
word_counts_dict=dict()
word_lists_dict=dict()
top_k_split=0.1 # 前 k%

for class_label in zero_shot_df['class_label'].unique():
    word_counts_dict[class_label]=zero_shot_df[zero_shot_df['class_label']==class_label]['cutted_content'].sum()
    word_counts_dict[class_label]=word_counts_dict[class_label].split(' ')
    word_counts_dict[class_label]=[x for x in word_counts_dict[class_label] if len(x)>1 and not x.isdigit()]
    word_counts_dict[class_label]=Counter(word_counts_dict[class_label])
    word_counts_dict[class_label]=sorted(word_counts_dict[class_label].items(), key=lambda x: x[1], reverse=True)
    len_words=len(word_counts_dict[class_label])
    word_counts_dict[class_label]=word_counts_dict[class_label][:int(top_k_split*len_words)] # NOTE 取前 k% 个
    
    word_lists_dict[class_label]=[x[0] for x in word_counts_dict[class_label]]
    print(class_label,len_words,len(word_lists_dict[class_label]))
    # break


教育 43403 4340
家居 40100 4010
时尚 32802 3280
时政 42269 4226
科技 26276 2627
房产 28334 2833
财经 29541 2954


In [4]:
feature_word_sets_dict=dict()
# 取 a-b (即差集)
a_split=0.08
b_split=1

for class_label in zero_shot_df['class_label'].unique():
    len_a=len(word_lists_dict[class_label])
    feature_word_sets_dict[class_label]=set(word_lists_dict[class_label][:int(len_a*a_split)])
    for class_label_2 in zero_shot_df['class_label'].unique():
        if class_label!=class_label_2:
            len_b=len(word_lists_dict[class_label_2])
            feature_word_sets_dict[class_label]=feature_word_sets_dict[class_label]-set(word_lists_dict[class_label_2][:int(len_b*b_split)]) # 差集
    print(class_label)
    print(feature_word_sets_dict[class_label])

教育
{'光明日报', '思政', '中职', '祖国', '校企', '院校', '责编', '开学', '副校长', '数学', '思政课', '北京师范大学', '立德', '高职', '该校', '育人', '论文', '师生'}
家居
{'马桶', '网址', '照明', '智家', '软装', '门窗', '电器', '欧派', '厨电', '甲醛', '洗碗机', '涂料', '沙发', '床垫', '无醛', '慕思', '洗衣机', '整装', '全屋', '烹饪', '美凯龙', '智能家居', '红星'}
时尚
{'表款', '身材', '圆形', '水晶', '美白', '连衣裙', '春夏', '针织', '玫瑰', '高跟鞋', '表带', '腕表', '女装', '时装周', '时髦', '迷人', '机芯', '衬衫', '西装', '镂空', '精华', '制表', '18K', '礼服', 'T恤', '痘痘', '表壳', '皮革', '飞轮', '时计', '指针', '时尚界', '腰带', '时装秀', '铂金', 'Vogue', '宝石', 'VOGUE', '外套', '毛衣', '大衣', '牛仔裤', '男装', '计时', '一页', 'Gucci', '参考价格', '克拉', '女人', '上衣', '时尚网', '精钢', '短裤', '镶嵌', '印花', '红毯', '风衣', '性感', '项链', '切割', '超模', '防晒', '毛孔', '时装', '蓝宝石'}
时政
{'线索', '职务', '腐败', '审查', '李克强', '巡察', '中央纪委', '巡视组', '党内', '纪委', '违纪', '查处', '问责', '执法', '党员干部', '纪检监察', '从严治党', '举报', '监委'}
科技
{'像素', '骁龙', '处理器', '图源', '魅族', '摄像头', '卖家', '页面', 'NFC', '新机', '高通', '刷新率', '后置', '一加', '转化率', '该机', 'realme', '直通车', '宝贝'}
房产
{'立享', '拨号', '房源', '住户', '地块', '公园', '建筑面积', '在售', '园林', '泰禾

In [5]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

# with open(kb_save_path, 'w',encoding='utf-8') as f:
#     f.write(json.dumps(feature_word_sets_dict,ensure_ascii=False, default=set_default)+'\n\n\n')
    